In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt

df=pd.read_csv('prices_aug24.csv', dtype={'TIME_M':object})
df.TIME_M = pd.to_datetime(df.TIME_M, format='%H:%M:%S.%f')
df.set_index('TIME_M', inplace = True)
df.head()


In [ ]:
df.head()
df_sec=df.groupby('SYM_ROOT')['PRICE'].resample('1s').mean().fillna(method = 'ffill').reset_index()

df_sec['moving_avg_10min'] = df_sec.groupby('SYM_ROOT')['PRICE'].rolling(600).mean().reset_index(0,drop=True).dropna()

In [ ]:
df_sec = df_sec.dropna()
df_sec.head()

In [ ]:
grouped=df.groupby('SYM_ROOT')    
    # create a temporary list to hold frames
frames = []
# iterate over the groups and apply moving average
for group in grouped.groups:
    frame = grouped.get_group(group)
    frame['price_ma5min'] = frame.rolling(600, on='SYM_ROOT').PRICE.mean()  #frame['PRICE'].rolling(600).mean()
    frames.append(frame)

    # concat the frames for a new dataframe
df_new = pd.concat(frames)

In [ ]:
def run_movingaverage(df):

    # sort the dataframe by the timestamp
   # df.sort_values('TIME_M', inplace=True)

    # group the df by stock, resample into 1 seconds mean prices, fill forward first, than backwards.
    grouped = df.groupby('SYM_ROOT')['PRICE'].resample('1s').mean().fillna(method = 'ffill').fillna(method = 'bfill')

    # create a temporary list to hold frames
    frames = []

    # iterate over the groups and apply exp. weighted moving average
    for group in grouped.groups:
        frame = grouped.get_group(group)
        frame['price_ma5min'] = frame['PRICE'].rolling('10M').mean()
        frames.append(frame)

    # concat the frames for a new dataframe
    df_new = pd.concat(frames)

    return df_new

%timeit df_new = run_movingaverage(df)

In [ ]:
df=pd.read_csv('prices_aug24.csv')
df.set_index('TIME_M', inplace = True)
df.groupby('SYM_ROOT')['PRICE'].apply(lambda x:x.rolling(center=False,window=2).mean())
df["moving average"] = df.groupby("person").apply(lambda x: x['PRICE'].ewm(span=60).mean())

In [ ]:
#Aggregate in one second, filling forward for missing values. 
grouped=df.groupby("SYM_ROOT").resample('1s').mean().fillna(method = 'ffill').fillna(method = 'bfill')
grouped.head()